In [1]:
%matplotlib inline


# Dağıtım için Vision Transformer Modelini Optimize Etme

[DeiT](https://ai.facebook.com/blog/data-efficient-image-transformers-a-promising-new-technique-for-image-classification)
Görüntü sınıflandırma için ImageNet üzerinde eğitilmiş bir Vision Transformer modelidir.

Bu eğitimde, öncelikle DeiT'nin ne olduğunu ve nasıl kullanılacağını ele alacağız,
sonra komut dosyası oluşturma, niceleme, optimize etme adımlarının tamamını uygulayacağız,
ve modeli iOS ve Android uygulamalarında nasıl kullanacağımıza bakacağız. Kuantize edilmiş, optimize edilmiş ve kuantize edilmemiş, optimize edilmemiş model performanslarını karşılaştıracağız.
Faydalarını göreceğiz.


## DeiT tam olarak nedir?

Evrişimli Sinir Ağları (CNN'ler), görü için ana modeller olmuştur.
2012'de derin öğrenme başladığından beri sınıflandırma problemlerinde temel olarak kullanılmıştır. Ancak CNN'ler
eğitim için yüz milyonlarca görüntüye ihtiyaç duyar.
`DeiT`, çok daha azını gerektiren bir görüntü dönüştürme modelidir.
Başarımı kanıtlanmış CNN mimarileriyle rekabet etmek için gereken eğitim/test verisi ve bilgi işlem kaynaklarının daha azına ihtiyaç duyar.
DeiT'nin temel bileşenleri:

-  Eğitimi çok daha büyük bir veri kümesinde simüle eden veri artırma;
-  Transformer network ağını öğrenen yerel bir CNN çıktısı.

`DeiT`, Transformers'ın bilgisayarlı görü problemlerine başarıyla uygulanabileceğini gösteriyor
Eğer veri ve kaynaklara sınırlı erişim var ise. Daha fazlası için
DeiT ile ilgili ayrıntılar burada [repo](https://github.com/facebookresearch/deit)
ve [makale](https://arxiv.org/abs/2012.12877).




## DeiT ile görüntü sınıflandırma

DebiT kullanarak görüntülerin nasıl sınıflandırılacağına ilişkin ayrıntılı bilgi için DieT deposundaki BENİOKU'yu takip edin veya hızlı bir test için önce gerekli paketleri kurun:



In [2]:
!pip install torch torchvision timm pandas requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.9 MB/s eta 0:00:00


In [3]:
!pip install timm pandas requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from PIL import Image
import torch
import timm
import requests
import torchvision.transforms as transforms
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

print(torch.__version__)
# should be 1.8.0


model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
model.eval()

transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
])

img = Image.open(requests.get("https://raw.githubusercontent.com/pytorch/ios-demo-app/master/HelloWorld/HelloWorld/HelloWorld/image.png", stream=True).raw)
img = transform(img)[None,]
out = model(img)
clsidx = torch.argmax(out)
print(clsidx.item())

1.13.1+cu116


Downloading: "https://github.com/facebookresearch/deit/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth" to /root/.cache/torch/hub/checkpoints/deit_base_patch16_224-b5f2ef4d.pth


  0%|          | 0.00/330M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


269


Artık sınıflandırmak için DeiT modelini kullanabileceğimizi doğruladığımıza göre
şimdi modelin iOS ve Android de çalışabilmesi için nasıl değiştirileceğini görelim.



## DeiT Komut dosyası
Modeli mobilde kullanmak için öncelikle şu komut dosyasını yazmamız gerekiyor:
modeli. [Komut Dosyası ve Optimize](https://pytorch.org/tutorials/recipes/script_optimized.html) bakın.
Kullanılan DeiT modelini dönüştürmek için aşağıdaki kodu çalıştırın.




In [5]:
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
model.eval()
scripted_model = torch.jit.script(model)
scripted_model.save("fbdeit_scripted.pt")

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main


Yaklaşık 346 MB boyutunda betikli model dosyasını fbdeit_scripted.pt
adı altında oluşturduk.



## Quantizing DeiT
Eğitimli model boyutunu ( 346 MB'lık fbdeit_scripted.pt model dosyamızı ) önemli ölçüde azaltmak için
çıkarım doğruluğunu yaklaşık olarak aynı tutarak, niceleme
modele uygulanır. DeiT'de kullanılan transformer modeli sayesinde
dinamik nicelemeyi modele kolayca uygulayabiliriz, çünkü dinamik
niceleme en iyi LSTM ve transformatör modelleri için çalışır ([ konu hakkında daha fazla bilgi için buraya bakın] (https://pytorch.org/docs/stable/quantization.html?highlight=quantization#dynamic-quantization).

Şimdi aşağıdaki kodu çalıştırın:




In [6]:
# Sunucu çıkarımı için "fbgemm" ve mobil çıkarım için "qnnpack" kullanın
backend = "fbgemm" # qnnpack ile değiştirildi, bu dizüstü bilgisayarda sayısallaştırılmış model için çok daha kötü çıkarım hızına neden oluyor
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend

quantized_model = torch.quantization.quantize_dynamic(model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8)
scripted_quantized_model = torch.jit.script(quantized_model)
scripted_quantized_model.save("fbdeit_scripted_quantized.pt")

/usr/local/lib/python3.8/dist-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Aynısını oluşturmak için ``scripted_quantized_model`` kullanabilirsiniz.
çıkarım sonucu:




In [7]:
out = scripted_quantized_model(img)
clsidx = torch.argmax(out)
print(clsidx.item())
# The same output 269 should be printed

269


## DeiT'i optimize etme
Kuantize edilmiş ve kodlanmış olanı kullanmadan önceki son adım
mobil cihazdaki modeli optimize etmektir.




In [8]:
from torch.utils.mobile_optimizer import optimize_for_mobile
optimized_scripted_quantized_model = optimize_for_mobile(scripted_quantized_model)
optimized_scripted_quantized_model.save("fbdeit_optimized_scripted_quantized.pt")

Oluşturulan fbdeit_optimized_scripted_quantized.pt dosyası, sayısallaştırılmış, kodlanmış, ancak optimize edilmemiş modelle yaklaşık olarak aynı boyuta sahiptir. Çıkarım sonucu aynı kalır.




In [11]:
out = optimized_scripted_quantized_model(img)
clsidx = torch.argmax(out)
print(clsidx.item())

269


## Lite Interpreter Kullanımı

Model boyutu küçültme ve çıkarımın Lite'ı ne kadar hızlandırdığını görmek için modelin lite sürümünü oluşturalım.




In [12]:
optimized_scripted_quantized_model._save_for_lite_interpreter("fbdeit_optimized_scripted_quantized_lite.ptl")
ptl = torch.jit.load("fbdeit_optimized_scripted_quantized_lite.ptl")

Lite model boyutu, lite olmayan sürümle karşılaştırılabilir olsa da, lite sürümünü mobilde çalıştırırken, çıkarım hızının artması beklenir.



## Çıkarım Hızını Karşılaştırma


In [13]:
with torch.autograd.profiler.profile(use_cuda=False) as prof1:
    out = model(img)
with torch.autograd.profiler.profile(use_cuda=False) as prof2:
    out = scripted_model(img)
with torch.autograd.profiler.profile(use_cuda=False) as prof3:
    out = scripted_quantized_model(img)
with torch.autograd.profiler.profile(use_cuda=False) as prof4:
    out = optimized_scripted_quantized_model(img)
with torch.autograd.profiler.profile(use_cuda=False) as prof5:
    out = ptl(img)

print("original model: {:.2f}ms".format(prof1.self_cpu_time_total/1000))
print("scripted model: {:.2f}ms".format(prof2.self_cpu_time_total/1000))
print("scripted & quantized model: {:.2f}ms".format(prof3.self_cpu_time_total/1000))
print("scripted & quantized & optimized model: {:.2f}ms".format(prof4.self_cpu_time_total/1000))
print("lite model: {:.2f}ms".format(prof5.self_cpu_time_total/1000))

original model: 765.54ms
scripted model: 700.83ms
scripted & quantized model: 650.25ms
scripted & quantized & optimized model: 519.33ms
lite model: 709.41ms


The following results summarize the inference time taken by each model
and the percentage reduction of each model relative to the original
model.




In [16]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'Model': ['original model','scripted model', 'scripted & quantized model', 'scripted & quantized & optimized model', 'lite model']})
df = pd.concat([df, pd.DataFrame([
    ["{:.2f}ms".format(prof1.self_cpu_time_total/1000), "0%"],
    ["{:.2f}ms".format(prof2.self_cpu_time_total/1000),
     "{:.2f}%".format((prof1.self_cpu_time_total-prof2.self_cpu_time_total)/prof1.self_cpu_time_total*100)],
    ["{:.2f}ms".format(prof3.self_cpu_time_total/1000),
     "{:.2f}%".format((prof1.self_cpu_time_total-prof3.self_cpu_time_total)/prof1.self_cpu_time_total*100)],
    ["{:.2f}ms".format(prof4.self_cpu_time_total/1000),
     "{:.2f}%".format((prof1.self_cpu_time_total-prof4.self_cpu_time_total)/prof1.self_cpu_time_total*100)],
    ["{:.2f}ms".format(prof5.self_cpu_time_total/1000),
     "{:.2f}%".format((prof1.self_cpu_time_total-prof5.self_cpu_time_total)/prof1.self_cpu_time_total*100)]],
    columns=['Çıkarım Süresi', 'Kesinti'])], axis=1)

print(df)

"""
        Model                             Çıkarım Süresi    Kesinti
0	original model                             1236.69ms           0%
1	scripted model                             1226.72ms        0.81%
2	scripted & quantized model                  593.19ms       52.03%
3	scripted & quantized & optimized model      598.01ms       51.64%
4	lite model                                  600.72ms       51.43%
"""

                                    Model Çıkarım Süresi Kesinti
0                          original model       765.54ms      0%
1                          scripted model       700.83ms   8.45%
2              scripted & quantized model       650.25ms  15.06%
3  scripted & quantized & optimized model       519.33ms  32.16%
4                              lite model       709.41ms   7.33%


'\n        Model                             Çıkarım Süresi    Kesinti\n0\toriginal model                             1236.69ms           0%\n1\tscripted model                             1226.72ms        0.81%\n2\tscripted & quantized model                  593.19ms       52.03%\n3\tscripted & quantized & optimized model      598.01ms       51.64%\n4\tlite model                                  600.72ms       51.43%\n'

### Daha fazlası için

- [Facebook Data-efficient Image Transformers](https://ai.facebook.com/blog/data-efficient-image-transformers-a-promising-new-technique-for-image-classification)_
- [Vision Transformer with ImageNet and MNIST on iOS](https://github.com/pytorch/ios-demo-app/tree/master/ViT4MNIST)_
- [Vision Transformer with ImageNet and MNIST on Android](https://github.com/pytorch/android-demo-app/tree/master/ViT4MNIST)_

